--- 
layout: single
title: CNN(RESNET)
use_math: true
comments: true

---

이 글에서는 2015년 ILSVRC에서 우승을 한 RESNET에 대해 다뤄보려고 한다. RESNET은 152개의 층을 갖는 알고리즘으로 이전에 소개한 VGG16 VGG19에 비해 엄청 깊어졌다. 그렇다면 네트워크의 깊이가 깊어지면 반드시 성능이 좋아지는지에 대해 의문이 들 것이다. 이를 확인하기 위해 RESNET 연구진들은 Convolution layer와 FC로 구성된 20층 56층의 네트워크를 각각 구성해 비교를 해보았다. 

![RESNET](http://whdbfla6.github.io/assets/images/resnet1.JPG)

이 그래프를 보면 56 layer의 네트워크가 20 layer의 네트워크보다 test error가 더 높은 것을 확인할 수 있다. 즉 depth가 깊어진다고 반드시 좋은 성능으로 이어지지 않는다는 말이다. 여기서 resnet 연구진들은 RESIDUAL BLOCK을 고안해 모델의 깊이는 깊어지면서 성능은 더욱 개선된 네트워크를 개발했다.

## Residual Block

우리가 기존에 알고 있는 네트워크들은 x값을 label에 mapping하는 방식으로 구성되었다. 즉 input image가 숫자 3에 해당한다면, x값이(각 픽셀 값) label3에 mapping하는 방식으로 말이다. 하지만 RESNET은 분류의 문제에 있어 y값은 x를 대변하는 것이라는 생각과 함께 x값을 레이블이 아닌 픽셀의 값(x)에 mapping하는 방식으로 변화를 주었다. 따라서 H(x)-y를 최소화 하는 방식에서 아래의 그림과 같이 H(x)-x를 최소화 하는 방식으로 학습을 진행한다.

$H(x)-x$ 최소화, $H(x)=x$

![RESNET](http://whdbfla6.github.io/assets/images/resnet2.JPG)

하지만 H(x)-x를 최소화하는 방식은 망이 엄청 깊어지는 경우 gradient vanishing 문제가 발생한다. resnet처럼 152개의 층이 존재하면, backpropagation을 진행할 수록 미분 값이 작아져 앞에 위치한 layer의 영향력이 작아진다는 것이다. 이를 해결하기 위해서 학습을 시킨 후에 x값을 더해 H(x)=F(x)+x가 x가 되도록 학습을 진행한다. 즉 F(x)가 0에 가까워지도록 학습하는 것이다. 이렇게 되면 미분 값이 $F'(x)+1$로 최소 미분 값이 1이 되어 gradient vanishing 문제를 해결해준다.

$H(x)-x$ 최소화 $ = F(x)+x-x$ 최소화, 즉 $F(x)$를 0에 가깝게 해라

![RESNET](http://whdbfla6.github.io/assets/images/resnet3.JPG)

## ResNet 코드 구성

이제부터 resnet50을 직접 코드로 작성해 볼 것이다. resnet50은 conv1x1과 conv3x3 연산이 포함된 여러개의 bottleneck으로 구성된다. 따라서 RESNET class를 정의하기 전에 conv3x3 conv1x1 bottleneck을 구성하기 위한 함수를 정의할 것이다. Bottle neck의 구성은 아래와 같다

![RESNET](http://whdbfla6.github.io/assets/images/resnet4.JPG)

In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [2]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

- conv3x3은 padding을 1로 설정하고 있으며, conv1x1은 padding이 없다

In [3]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes) #conv1x1(64,64)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)#conv3x3(64,64)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion) #conv1x1(64,256)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x) # 1x1 stride = 1
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = stride 
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 stride = 1
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

`forward(self, x)`부분을 살펴보면 convolution 연산을 3번 수행하는 것을 확인할 수 있다. `self.conv1(x)` ` self.conv3(out)`은 conv1x1을 진행하며 stide값은 기본값 1로 고정되어 있는 반면 `self.conv2(out)`은 conv3x3을 수행하며 stride 값은 원하는 값을 넣을 수 있다. 모든 layer를 통과한 후에 `out += identity`을 통해 x값을 더해주게 된다. 중요한 것은 layer를 통과 한 후의 F(x)의 size와 input x의 차원이 같아야 덧셈 연산을 진행할 수 있다는 것인데, `self.downsample(x)`를 통해 차원을 동일시할 수 있다.

In [4]:
class ResNet(nn.Module):
    # model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) #resnet 50 
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        
        self.inplanes = 64
               
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        
        downsample = None
        
        if stride != 1 or self.inplanes != planes * block.expansion: 
            
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride), #conv1x1(256, 512, 2)
                nn.BatchNorm2d(planes * block.expansion), #batchnrom2d(512)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes * block.expansion #self.inplanes = 128 * 4
        
        for _ in range(1, blocks): 
            layers.append(block(self.inplanes, planes)) # * 3

        return nn.Sequential(*layers)
    

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [1]:
def resnet50(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) #=> 3*(3+4+6+3) +(conv1) +1(fc) = 48 +2 = 50
    return model

In [6]:
res = resnet50()
res

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

참고문헌<br/>
[모두를위한딥러닝시즌2](https://deeplearningzerotoall.github.io/season2/)<br/>
[프라이데이](https://ganghee-lee.tistory.com/41)<br/>
[b스카이비전](https://bskyvision.com/644)